In [33]:
# detControlMatExistence
# how to concatenate vectors in a loop: http://akuederle.com/create-numpy-array-with-for-loop

import numpy as np
import scipy.linalg as LA

def detControlMatExistence(A, B, indicMat):
    n=int(len(indicMat)/6) # indicMat is 6n x 6n

# Compute good (Fp,Fq) sample space
    #Fq_range=np.arange(-2, 4, 0.2).tolist()
    #Fp_range=np.arange(-0.1, 0.1, 0.015).tolist()
    Fq_range=np.arange(-1, 1, 0.5).tolist()
    Fp_range=np.arange(-2, 2, 0.5).tolist()
    
# Initialize arrays, will be populated in loops
    feas=False # boolean
    numfeas,myCosts= (np.empty((0,1)) for i in range(2))
    feasFs,myFbases=(np.empty((0,2)) for i in range(2)) # 0 for dim to concatenate on, 2 for length of vectors being concatenated
    dataFull, data_zero = (np.array([]) for i in range(2)) # for each config
    CLmat=np.empty((6*n,6*n))

    for Fq in Fq_range:
        for Fp in Fp_range:
            if not(Fq==0 or Fp==0): # skip iteration if either zero
                #print("(Fq,Fp)=",Fq,",",Fp,")")
                F=assignF(Fp,Fq,indicMat)
                CLmat=A-np.dot(B,F) # CLmat=A-BF
                eigs,evecs=LA.eig(CLmat) # closed loop eigenvalues
                eigMags=np.absolute(eigs)
              #  if all(np.absolute(eigs)<=np.ones(6*n,1)):
                if all(eigMags<=1): 
                    feas=True
                    feasFs=np.append(feasFs,[[Fp, Fq]],axis=0)
                val=np.sum(eigMags[np.where(eigMags > 1)])
                myCosts=np.append(myCosts,[[val]],axis=0) # temp
                myFbases=np.append(myFbases,[[Fp, Fq]],axis=0)

    if feas==True:
        print("Config feasible!")
        numfeas=np.append(numfeas,[[len(feasFs)]],axis=0) # number of rows
    else:
        feas=False
        print("No F found for config --> non-conclusive")
        numfeas=np.append(numfeas,[[0]],axis=0)

    dataFull=np.concatenate((myFbases,myCosts),axis=1) # [Fp Fq myCost]  
    print("[Fp,Fq,myCost]=\n",dataFull)
    numTried=len(dataFull) # number of rows
    num_act=np.count_nonzero(indicMat)/2
       
    # return feas,feasFs,num_act,numfeas,numTried
    return feas,feasFs,numfeas,numTried,num_act

In [34]:
def assignF(Fp,Fq,indicMat):
    n=int(len(indicMat)/6) # indicMat is 6n x 6n
    
    Hblock1=np.zeros((3*n,3*n))   
    ridx,colidx=np.nonzero(indicMat[0:3*n,0:3*n]) # python indexing goes first to (last-1) 
    Hblock1[ridx,colidx]=Fq

    Hblock2=np.zeros((3*n,3*n))   
    ridx,colidx=np.nonzero(indicMat[0:3*n,0:3*n]) 
    Hblock2[ridx,colidx]=Fp
  
    upper=np.concatenate((Hblock1, np.zeros((3*n,3*n))),axis=1)
    lower=np.concatenate((np.zeros((3*n,3*n)),Hblock2),axis=1)
    F=np.concatenate((upper,lower))
    #                    concat_XR = np.concatenate((X, R), axis = 1)
    #concat_XR_halfs = np.concatenate(((-1/2) * R, (1/2) * X), axis = 1)
    #B = np.concatenate((concat_XR, concat_XR_halfs))
    #F=np.identity(6*n)
    
    print(F)
    #print("Size of F=",F.shape)
    return F

# MATLAB:
#function F = assignF(Fp,Fq,IndicMat)
#   n=size(IndicMat,1)/6; %IndicMat is 6nx6n
#   idx=find(IndicMat(1:3*n,1:3*n)~=0);
 #  Hvec=zeros(3*n*3*n,1); Hvec(idx)=Fp; Hblock1=reshape(Hvec,3*n,3*n); 
 #  Hvec=zeros(3*n*3*n,1); Hvec(idx)=Fq; Hblock2=reshape(Hvec,3*n,3*n); 
 #  F=[Hblock1 zeros(3*n,3*n); zeros(3*n,3*n) Hblock2];
#end

In [37]:
n=1 # Temp
A = np.identity(6*n)
sz=(6*n,6*n)
B=np.ones(sz) # TEMP: replace with call to createRXmatrices_3ph
indicMat=np.identity(6*n) # TEMP: replace with actual config you want to evaluate

MYfeas,MYfeasFs,MYnumfeas,MYnumTried,MYnumact=detControlMatExistence(A, B, indicMat)
print("--------------------- Outputs ----------------------")
print("Myfeas=",MYfeas)
print("MyfeasFs=\n",MYfeasFs)
print("Mynumfeas=",MYnumfeas)
print("MynumTried=",MYnumTried)
print("Mynumact=",MYnumact)

[[-1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -2.  0.  0.]
 [ 0.  0.  0.  0. -2.  0.]
 [ 0.  0.  0.  0.  0. -2.]]
[[-1.   0.   0.   0.   0.   0. ]
 [ 0.  -1.   0.   0.   0.   0. ]
 [ 0.   0.  -1.   0.   0.   0. ]
 [ 0.   0.   0.  -1.5  0.   0. ]
 [ 0.   0.   0.   0.  -1.5  0. ]
 [ 0.   0.   0.   0.   0.  -1.5]]
[[-1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.]]
[[-1.   0.   0.   0.   0.   0. ]
 [ 0.  -1.   0.   0.   0.   0. ]
 [ 0.   0.  -1.   0.   0.   0. ]
 [ 0.   0.   0.  -0.5  0.   0. ]
 [ 0.   0.   0.   0.  -0.5  0. ]
 [ 0.   0.   0.   0.   0.  -0.5]]
[[-1.   0.   0.   0.   0.   0. ]
 [ 0.  -1.   0.   0.   0.   0. ]
 [ 0.   0.  -1.   0.   0.   0. ]
 [ 0.   0.   0.   0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.5  0. ]
 [ 0.   0.   0.   0.   0.   0.5]]
[[-1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  